# Transfer learning

## Fine tuning

In [3]:
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
from torch.optim import SGD

weights = ResNet18_Weights.DEFAULT
preprocessing = weights.transforms()  # incluye el data augmentation realizado (resize and crop).
# original_classes_names = weights.meta['categories']

net = resnet18(weights=weights)

# Sustitución del clasificador:
n_classes = 10
num_ft = net.fc.in_features
net.fc = nn.Linear(num_ft, n_classes)

# Optimizador (todos los parámetros serán actualizados):
optimizer = SGD(net.parameters(), lr=1e-3, momentum=0.9)

## Freezing

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
from torch.optim import SGD

weights = ResNet18_Weights.DEFAULT
preprocessing = weights.transforms()
# original_classes_names = weights.meta['categories']

net = resnet18(weights=weights)

# Congelar todos los parámetros:
for param in net.parameters():
    param.requires_grad = False

# Sustitución del clasificador:
n_classes = 10
num_ft = net.fc.in_features
net.fc = nn.Linear(num_ft, n_classes)  # por default, requires_grad = True.

# Optimizador (solo los parámetros del clasificador final se actualizarán):
optimizer = SGD(net.fc.parameters(), lr=1e-3, momentum=0.9)

In [ ]:
# Cuando se están actualizando varias capas o fragmentos de un Sequential se deben guardar
# en una lista los parámetros que se actualizarán (solo los que tiene requires_grad = True):

print('Parámetros que se actualizarán:')
params_to_update = []
for name, param in net.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print('\t', name)

optimizer = SGD(params_to_update, lr=1e-3, momentum=0.9)

## Otras arquitecturas

In [ ]:
# AlexNet:
# El clasificador (net.classifier) es un Sequential de módulos FC. El último módulo (6) es el clasificador final.
# Por lo tanto, se debe realizar lo siguiente:
net.classifier[6] = nn.Linear(4096, n_classes)

# VGG:
# Ocurre lo mismo que en AlexNet, la capa FC final está en la posición 6:
net.classifier[6] = nn.Linear(4096, n_classes)

# SqueezeNet (1.0):
# El clasificador es un Sequential de dropout -> conv2d (clasificador) -> relu -> avgpool.
# Por lo tanto, se debe modificar la capa convolucional:
net.classifier[1] = nn.Conv2d(512, n_classes, kernel_size=(1,1), stride=(1,1))

# DenseNet-121:
# El clasificador es un módulo simple (no Sequential), por lo tanto:
net.classifier = nn.Linear(1024, n_classes)

# Inception v3:
# Contiene dos salidas por lo que se deben modificar dos capas:
net.AuxLogits.fc = nn.Linear(768, n_classes)
net.fc = nn.Linear(2048, n_classes)

# Solo se usa la salida final en testing. Para training, se deben ponderar las losses.
# Se debe modificar el trainer agregando el siguiente fragmento:
if is_inception and mode == 'train':
    output, aux_output = net(x)
    loss1 = loss_fn(output, y)
    loss2 = loss_fn(aux_output, y)
    loss = loss1 + 0.4*loss2

# En general, basta imprimir el modelo para saber qué capas deben ser modificadas.